In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# Load the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

g = sns.countplot(Y_train)

In [ ]:
#test for null
X_train.isnull().any().sum()

In [ ]:
#test for null
test.isnull().any().sum()

In [ ]:
#normalize data (grey-scale)
X_train = X_train / 255.0

test = test / 255.0

In [ ]:
#reshape
X_train = X_train.values.reshape(-1, 28,28,1)

test = test.values.reshape(-1, 28,28,1)

print(X_train.shape)
print(test.shape)

In [ ]:
plt.imshow(X_train[0][:,:,0])

In [ ]:
#Label encoder
Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
#Train/test split
X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.4, random_state = 2018)


In [ ]:
#Set up model

model = Sequential()

#Add layers
model.add(Conv2D(10, kernel_size = 2, activation = 'relu', input_shape = (28,28,1)))

model.add(Conv2D(10, kernel_size = 2, activation = 'relu'))

#Flatten
model.add(Flatten())

#Add output layer
model.add(Dense(10, activation = 'softmax'))




In [ ]:
#Compile
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#fitting
training = model.fit(X_train, y_train, validation_split = 0.2, epochs = 3)

In [ ]:
#plot learning curves
# Extract the history from the training object
history = training.history

# Plot the training loss 
plt.plot(history['loss'])
# Plot the validation loss
plt.plot(history['val_loss'])

# Show the figure
plt.show()


In [ ]:
#evaluate

model.evaluate(X_val, y_val)

In [ ]:
#Predict
prediction = model.predict(test)

prediction = np.argmax(prediction, axis = 1)

prediction = pd.Series(prediction, name = "Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),prediction],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)